## Data Scraping
Data scraping is a technique in which a computer program extracts data from human-readable output coming from another program.

Beautiful Soup (called bs4 when calling the library in Python) is a Python package for parsing HTML and XML documents. This will be used to extract data from HTML pages.
Urllib is a Python package that collects several modules for working with URLs:
* request
* error
* parse
* robot parser

The following code block loads these packages and imports BeautifulSoup as soup, which means that we can use "soup" when calling BeautifulSoup functions instead of "BeautifulSoup" to simplify the code somewhat. Also, Urllib as "uReq" because the code is only importing the request module.

In [ ]:
from bs4 import BeautifulSoup as soup  # Library for HTML data structures
from urllib.request import urlopen as uReq  # Library for opening URLs

Next a variable is created to store the website URL of interest. The example used here is newegg.com for computer central processing unit (CPUs) products:

In [ ]:
page_url = "https://www.newegg.com/p/pl?Submit=StoreIM&Category=34&Depa=1"